In [ ]:
SEED = 9

import os
os.environ['PYTHONHASHSEED'] = str(SEED)

import random
random.seed(SEED)

import numpy as np
rng = np.random.default_rng(SEED) 
ss = rng.bit_generator._seed_seq # use for multiprocessing

from neuralprophet import set_random_seed
set_random_seed(SEED)

SEED_states_log = []


# -------------------------------------------------------------------------------------
import sys
print(sys.version)
import collections
from pprint import pprint as pp
from functools import reduce
import multiprocessing
import concurrent.futures
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statistics import mean, median, mode
from scipy import stats
from scipy.special import boxcox1p, inv_boxcox1p
import statsmodels.api as sm



tickers = (
"BTC",
"ETH",
)

pov = (
# 'Astro'
# 'Astro_Geo'
'Astro_Helio'
)

usdpairs = tickers

# usdpairs = (
# "BTC",
# "RealEstate",
# "S&P",
# "Crude",
# "Gold",
# "Silver",
# "Dollar",
# "Bond",
# "Volatility",
# )

maxint = 9

def vortex(n):
#     return sum([int(x) for x in list(str(abs(int(float(str(n).replace(".",""))))))])
    return sum([int(x) for x in list(str(abs(int(float(n)))))])

def blackhole(n):
    if n > 0:
        if vortex(n) > maxint:
            return blackhole(vortex(n))
        if vortex(n) <= maxint:
            return vortex(n)
    else:
        if vortex(n) > maxint:
            return blackhole(vortex(n))*-1
        if vortex(n) <= maxint:
            return vortex(n)*-1

def log_transform(df):
    # copy the dataframe
    tdf = df.copy()
    # apply log scaling
    for column in tdf.columns:
        tdf[column] = np.log(tdf[column])

    tdf.replace([np.inf, -np.inf], np.nan, inplace=True)
    tdf = tdf.astype(float)
    tdf = tdf.interpolate(method='linear', axis=0).ffill().bfill()
#     tdf = tdf.dropna(axis='columns')
    tdf = tdf.fillna(0)

    return tdf


def boxcox(df):
    # copy the dataframe
    _df = df.copy()

    # apply log scaling
    for column in _df.columns:

#         tdf[column] = stats.boxcox(tdf[column])
        _df[column] = boxcox1p(_df[column], 0.25)

#     _df = _df.dropna(axis='columns')
    _df = _df.fillna(0)
    _df = _df.astype(float)

    return _df

def inv_boxcox(df):
    # copy the dataframe
    _df = df.copy()

    # apply log scaling
    for column in _df.columns:

#         tdf[column] = stats.boxcox(tdf[column])
        _df[column] = inv_boxcox1p(_df[column], 0.25)

#     _df = _df.dropna(axis='columns')
    _df = _df.fillna(0)
    _df = _df.astype(float)

    return _df


def min_max_scaling(df):
    # copy the dataframe
    tdf = df.copy()
    # apply min-max scaling
    for column in tdf.columns:
        tdf[column] = (tdf[column] - tdf[column].min()) / (tdf[column].max() - tdf[column].min())
#     tdf = tdf.dropna(axis='columns')
    tdf = tdf.fillna(0)
    return tdf


def dfta(ticker):

    path = '///'
    
    if ticker in usdpairs:
        # Loading the dataset
        file = ticker + ".csv"
        url = path + file
        df = pd.read_csv(url,parse_dates = True,index_col=0)
    else:
        file = "BTC.csv"
        url = path + file
        c = pd.read_csv(url,parse_dates = True,index_col=0)
        c = c.rename(columns={"open": "o",
                              "high": "h",
                              "low": "l",
                              "close": "c"})

        # Loading the dataset
        file = ticker + ".csv"
        url = path + file
        df = pd.read_csv(url,parse_dates = True,index_col=0)

        df = pd.merge(df, c,left_index=True,right_index=True)



        df["open"] = df["open"].div(df["o"])
        df["high"] = df["high"].div(df["h"])
        df["low"] = df["low"].div(df["l"])
        df["close"] = df["close"].div(df["c"])

        df = df.drop([
              'o',
              'h',
              'l',
              'c',
             ] , axis='columns')


#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    df['y'] = df[["open", "close"]].sum(axis=1).div(2) # (o+c)/2

    df = boxcox(df)
    df = log_transform(df)
    df = boxcox(df)
# ----------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------

# ----------------------------------------------------------------------------------------------------
#######################################################################################################

#     start_date = "1999-01-01"
#     end_date = "2020-01-01"
# #     end_date = "2020-07-19"
# #     end_date = "2020-08-27"
# #     end_date = "2021-01-01"
# #     end_date = "2021-05-01"
#
#     after_start_date = df.index >= start_date
#     before_end_date = df.index <= end_date
#     between_two_dates = after_start_date & before_end_date
#     df = df.loc[between_two_dates]

#######################################################################################################
# ----------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------
    sun = '///SN_d_tot_V2.0.csv'
    sf = pd.read_csv(sun,parse_dates = True,index_col=0)
    df = pd.merge(df, sf,left_index=True,right_index=True)

    for _ in sf:
        df["d_" + _ ] = df["y"].div(df[["y", _]].sum(axis=1))
# ----------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------
    ki = '///k-indices.csv'
    kf = pd.read_csv(ki,parse_dates = True,index_col=0)
    df = pd.merge(df, kf,left_index=True,right_index=True)

    for _ in kf:
        df["d_" + _ ] = df["y"].div(df[["y", _]].sum(axis=1))
# ----------------------------------------------------------------------------------------------------
# ----------------------------------------------------------------------------------------------------
    file = pov + ".csv"
    url = '///' + file
    astro = pd.read_csv(url,parse_dates = True,index_col=0)
    astro = astro[~astro.index.duplicated()]
#     astro.columns=['ch' + str(x) for x in range(len(astro.columns))]
    df = pd.merge(df, astro,left_index=True,right_index=True)

    for _ in astro:
        df["d_" + _ ] = df["y"].div(df[["y", _]].sum(axis=1))
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
    df = df.tail(1884)
    df = df.head(1883)

#     df = df.tail(2001)
#     df = df.head(2000)

    print(ticker)
    print(len(df))
#@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

    df['ds'] = df.index.copy()

    earliest = df.iloc[:1]['ds']
    earliest = pd.to_datetime(earliest).reset_index(drop=True)[0]
    earliest = earliest.strftime('%Y-%m-%d')

    latest = df.iloc[-1:]['ds']
    latest = pd.to_datetime(latest).reset_index(drop=True)[0]
    latest = latest.strftime('%Y-%m-%d')

    print(earliest,latest)

    # gap check
    all_dates = pd.date_range(start=earliest, end=latest)
    complete_dates = list(all_dates.strftime('%Y-%m-%d'))
    questionable_dates = list(df["ds"])

    for i in complete_dates:
        if i not in questionable_dates:
            pass
        else:
            print("%%%%%%%%%%%%%%%%%%%%%% PROBLEM WITH DATE GAP %%%%%%%%%%%%%%%%%%%%%%")

    df = df.drop([
      'ds',
     ] , axis='columns')

#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    print(df.columns[df.isna().any()].tolist())
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    tdf = df.applymap(lambda x: blackhole(x))
    tdf.columns=['z' + str(x) for x in range(len(tdf.columns))]
#     tdf = tdf.dropna(axis='columns')
    tdf = tdf.fillna(0)
    df = pd.merge(df, tdf,left_index=True,right_index=True)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
#     df = boxcox(df)
#     df = log_transform(df)
    df = min_max_scaling(df)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    df = df.loc[:,df.apply(pd.Series.nunique) > 2]

    path = '///'
    file = ticker + "Backup.csv"
    out = path + file
    df.to_csv(out,index=False)

    file = "_" + ticker + ".csv"
    out = path + file
    df.to_csv(out,index=True)
    print(len(df.columns))


#     fig, ax = plt.subplots(figsize=(16,9))
#     df["y"].plot(xlabel="\nCreated on "+latest, ylabel="Scale\n", ax=ax)
#     ax.xaxis.label.set_size(20)
#     ax.yaxis.label.set_size(20)
#     ax.tick_params(axis='both', which='major', labelsize=15)
#     ax.set_title(ticker, fontsize=20, fontweight="bold")

# dfta("BTC")

with concurrent.futures.ProcessPoolExecutor() as executor:
# with concurrent.futures.ThreadPoolExecutor() as executor:
    executor.map(dfta,tickers)
